In [1]:
import torch

def find_parameters(model, seen=tuple()):
    """
    Recursively explore a PyTorch model and find all Tensors
    """
    seen = seen + (id(model),)

    parameters = []
    for param in dir(model):
        item = getattr(model, param)
        if isinstance(item, torch.Tensor):
            parameters.append((model, param, item))
        elif isinstance(item, torch.nn.Module) and id(item) not in seen:
            parameters += find_parameters(item, seen)
            seen = seen + (id(item),)

    return parameters

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
import numpy as np
import onnx
import onnxruntime  as ort
from vnnlib.compat import read_vnnlib_simple

def load_vnnlib(model_path="../onnx/14_ieee_ml4acopf.onnx", vnnlib_path="../vnnlib/14_ieee_prop1.vnnlib"):
    # Load the ONNX model into memory
    model = onnx.load(model_path)
    sess_options = ort.SessionOptions()
    sess_options.intra_op_num_threads = 1
    sess = ort.InferenceSession(model_path, sess_options=sess_options)

    # Get information about the input and output nodes of the ONNX model
    input_info = sess.get_inputs()
    output_info = sess.get_outputs()
    input_name = input_info[0].name
    input_shape = input_info[0].shape
    output_name = output_info[0].name
    output_shape = output_info[0].shape

    # Load the vnnlib file into memory
    spec, = read_vnnlib_simple(vnnlib_path, input_shape[1], output_shape[1])

    return input_shape, spec

In [3]:
import pandas as pd
import glob

instances = pd.read_csv('../instances.csv', names=['onnx', 'vnnlib', 'runtime'])

In [4]:
import onnx2torch

for i, row in instances.iterrows():
    model_path = '../' + row['onnx']
    vnnlib_path = '../' + row['vnnlib']

    # Load the onnx model into PyTorch
    torch_model = onnx2torch.convert(model_path)

    # Turn all float32 parameters into float64 tensors
    for obj, param, item in find_parameters(torch_model):
        if item.dtype == torch.float32:
            setattr(obj, param, type(item)(item.to(torch.float64)))

    # Load the vnnlib spec
    input_shape, (input_spec, output_spec) = load_vnnlib(model_path, vnnlib_path)
    input_spec = torch.tensor(input_spec, dtype=torch.float64)

    # Create a dummy input
    X = input_spec[None, :, 0]
    assert list(X.shape) == input_shape

    input_names = ['X']
    output_names = ['Y']

    # Export the model
    torch.onnx.export(torch_model, X, model_path + '.64bit', input_names=input_names, output_names=output_names, opset_version=12)

/opt/conda/envs/mega/lib/python3.10/site-packages/vnnlib/parser.py:276: UserWarning: literal negation does not strictly follow SMT-LIB
  warnings.warn("literal negation does not strictly follow SMT-LIB")
/opt/conda/envs/mega/lib/python3.10/site-packages/onnx2torch/node_converters/slice.py:33: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  axes = axes.detach().cpu().numpy()
/opt/conda/envs/mega/lib/python3.10/site-packages/onnx2torch/node_converters/slice.py:38: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  steps = steps.detach().cpu().numpy()
/opt/conda/envs/